In [2]:
import pandas as pd
import matplotlib.pyplot as pyplot

In [3]:
# Read VGG16 results data
results_path = "./../results/vgg16/results.csv"
results_df = pd.read_csv(results_path, sep = ";")

In [4]:
results_df.head()

,generator_type,images_per_class_num,images_size,batch_size,fc1,fc2,training_time,train_loss_last_epoch,train_loss_avg5,train_loss_avg10,...,test_sparse_categorical_accuracy,true_3_pred_3,true_3_pred_5,true_3_pred_7,true_5_pred_3,true_5_pred_5,true_5_pred_7,true_7_pred_3,true_7_pred_5,true_7_pred_7
0,v1,90,64,32,16,16,97.293156,0.038325,0.042282,0.046802,...,0.992593,90,0,0,0,90,0,0,2,88
1,v1,90,64,32,256,256,102.302117,0.004976,0.005383,0.005956,...,0.992593,90,0,0,0,90,0,0,2,88
2,v1,90,64,32,4096,4096,133.971879,0.000311,0.000368,0.000451,...,0.992593,90,0,0,0,90,0,0,2,88
3,v1,90,128,32,16,16,372.864259,0.010527,0.011262,0.012402,...,1.000000,90,0,0,0,90,0,0,0,90
4,v1,90,128,32,256,256,386.086895,0.000350,0.000376,0.000404,...,0.996296,90,0,0,0,90,0,0,1,89


In [32]:
# Best models based on test accuracy, val accuracy (last epoch) and train accuracy (last epoch)
sort_cols = ["test_sparse_categorical_accuracy", "val_sparse_categorical_accuracy_last_epoch", "train_sparse_categorical_accuracy_last_epoch"]
results_df.loc[:, list(results_df.columns[0:6]) + sort_cols].sort_values(by = sort_cols, ascending = False).reset_index(drop = True)

,generator_type,images_per_class_num,images_size,batch_size,fc1,fc2,test_sparse_categorical_accuracy,val_sparse_categorical_accuracy_last_epoch,train_sparse_categorical_accuracy_last_epoch
0,v1,90,128,32,16,16,1.000000,1.000000,1.000000
1,v1,90,256,32,4096,4096,1.000000,1.000000,1.000000
2,v1,270,64,32,16,16,1.000000,1.000000,1.000000
3,v1,270,128,32,16,16,1.000000,1.000000,1.000000
4,v1,270,128,32,256,256,1.000000,1.000000,1.000000
5,v1,270,128,32,4096,4096,1.000000,1.000000,1.000000
6,v1,270,256,32,256,256,1.000000,1.000000,1.000000
7,v1,270,256,32,4096,4096,1.000000,1.000000,1.000000
8,v1,900,128,32,256,256,1.000000,1.000000,1.000000
9,v1,900,128,32,4096,4096,1.000000,1.000000,1.000000


In [47]:
# Best models based on test accuracy, val accuracy (last epoch) and train accuracy (last epoch)
sort_cols = ["test_sparse_categorical_accuracy", "val_sparse_categorical_accuracy_last_epoch", "train_sparse_categorical_accuracy_last_epoch"]
results_df.loc[:, [col for col in results_df.columns if not (col.startswith("true") or "loss" in col or "time" in col)]].sort_values(by = sort_cols, ascending = False).reset_index(drop = True)

,generator_type,images_per_class_num,images_size,batch_size,fc1,fc2,train_sparse_categorical_accuracy_last_epoch,train_sparse_categorical_accuracy_avg5,train_sparse_categorical_accuracy_avg10,val_sparse_categorical_accuracy_last_epoch,val_sparse_categorical_accuracy_avg5,val_sparse_categorical_accuracy_avg10,test_sparse_categorical_accuracy
0,v1,90,128,32,16,16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,v1,90,256,32,4096,4096,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,v1,270,64,32,16,16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,v1,270,128,32,16,16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,v1,270,128,32,256,256,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,v1,270,128,32,4096,4096,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,v1,270,256,32,256,256,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,v1,270,256,32,4096,4096,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
8,v1,900,128,32,256,256,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,v1,900,128,32,4096,4096,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
